## Importando bibliotecas

In [1]:
import cv2
import numpy as np
import os
import time
import matplotlib.pyplot as plt

## Configurando caminhos e carregando arquivos do modelo treinado

In [2]:
labels_path = os.path.sep.join(['darknet/cfg/coco.names'])
LABELS = open(labels_path).read().strip().split("\n")

In [3]:
weights_path = os.path.sep.join(['darknet/yolov4.weights'])
config_path = os.path.sep.join(['darknet/cfg/yolov4.cfg'])

## Definindo configurações para a detecção 

In [4]:
net = cv2.dnn.readNet(config_path, weights_path)

In [5]:
np.random.seed(42)
COLORS = np.random.randint(0, 255, size=(len(LABELS), 3), dtype="uint8")

In [6]:
ln = net.getLayerNames()
print("Todas as camadas (layers):")
print(ln)
print("Total: "+ str(len(ln)))
print("Camadas de saída: ")
print(net.getUnconnectedOutLayers())
ln = [ln[i[0] - 1] for i in net.getUnconnectedOutLayers()]
print(ln)

Todas as camadas (layers):
['conv_0', 'bn_0', 'mish_1', 'conv_1', 'bn_1', 'mish_2', 'conv_2', 'bn_2', 'mish_3', 'identity_3', 'conv_4', 'bn_4', 'mish_5', 'conv_5', 'bn_5', 'mish_6', 'conv_6', 'bn_6', 'mish_7', 'shortcut_7', 'conv_8', 'bn_8', 'mish_9', 'concat_9', 'conv_10', 'bn_10', 'mish_11', 'conv_11', 'bn_11', 'mish_12', 'conv_12', 'bn_12', 'mish_13', 'identity_13', 'conv_14', 'bn_14', 'mish_15', 'conv_15', 'bn_15', 'mish_16', 'conv_16', 'bn_16', 'mish_17', 'shortcut_17', 'conv_18', 'bn_18', 'mish_19', 'conv_19', 'bn_19', 'mish_20', 'shortcut_20', 'conv_21', 'bn_21', 'mish_22', 'concat_22', 'conv_23', 'bn_23', 'mish_24', 'conv_24', 'bn_24', 'mish_25', 'conv_25', 'bn_25', 'mish_26', 'identity_26', 'conv_27', 'bn_27', 'mish_28', 'conv_28', 'bn_28', 'mish_29', 'conv_29', 'bn_29', 'mish_30', 'shortcut_30', 'conv_31', 'bn_31', 'mish_32', 'conv_32', 'bn_32', 'mish_33', 'shortcut_33', 'conv_34', 'bn_34', 'mish_35', 'conv_35', 'bn_35', 'mish_36', 'shortcut_36', 'conv_37', 'bn_37', 'mish_38'

In [7]:
#def mostrar(img):
  #fig = plt.gcf()
  #fig.set_size_inches(16, 10)
  #plt.axis("off")
  #plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
  #plt.show()

In [8]:
def blob_imagem(net, imagem, mostrar_texto=True):
  inicio = time.time() 
  blob = cv2.dnn.blobFromImage(imagem, 1 / 255.0, (416, 416), swapRB=True, crop=False)
  net.setInput(blob)
  layerOutputs = net.forward(ln)
  termino = time.time()
  if mostrar_texto:
    print("YOLO levou {:.2f} segundos".format(termino - inicio))
  return net, imagem, layerOutputs

In [9]:
def deteccoes(detection, _threshold, caixas, confiancas, IDclasses):
  scores = detection[5:] 
  classeID = np.argmax(scores)  
  confianca = scores[classeID]

  if confianca > _threshold:
      caixa = detection[0:4] * np.array([W, H, W, H])     
      (centerX, centerY, width, height) = caixa.astype("int")
            
      x = int(centerX - (width / 2))
      y = int(centerY - (height / 2))

      caixas.append([x, y, int(width), int(height)])
      confiancas.append(float(confianca))
      IDclasses.append(classeID)
      
  return caixas, confiancas, IDclasses

In [10]:
def funcoes_imagem(imagem, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=True):  
  (x, y) = (caixas[i][0], caixas[i][1])
  (w, h) = (caixas[i][2], caixas[i][3])

  cor = [int(c) for c in COLORS[IDclasses[i]]]
  cv2.rectangle(imagem, (x, y), (x + w, y + h), cor, 2) 
  texto = "{}: {:.4f}".format(LABELS[IDclasses[i]], confiancas[i])
  if mostrar_texto:
    print("> " + texto)
    print(x,y,w,h)
  cv2.putText(imagem, texto, (x, y - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.5, cor, 2)

  return imagem,x,y,w,h

In [11]:
def pega_centro(x,y,w,h):
    x1 = int(w/2)
    y1 = int(h/2)
    cx = x + x1
    cy = y + y1
    return cx, cy

In [12]:
pos_linha = 600
offset = 8

___

## Lendo arquivo

In [13]:
video = 'video/Video1.mp4'
cap = cv2.VideoCapture(video)
conectado, video = cap.read()

In [14]:
conectado

True

In [15]:
video.shape

(720, 1280, 3)

In [16]:
video_largura = video.shape[1]
video_altura = video.shape[0]
video_largura, video_altura

(1280, 720)

___

## Redimensionamento do tamanho do video (opcional)

In [17]:
def redimensionar(largura, altura, largura_maxima = 600): 
  if (largura > largura_maxima):
    proporcao = largura / altura
    video_largura = largura_maxima
    video_altura = int(video_largura / proporcao)
  else:
    video_largura = largura
    video_altura = altura

  return video_largura, video_altura

In [18]:
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [19]:
amostras_exibir = 20
amostra_atual = 0

## Configurações de vídeo de saída e codec

In [20]:
nome_arquivo = 'resultado.avi'
fourcc = cv2.VideoWriter_fourcc(*'XVID') # MP4V

In [21]:
fps = 24

In [22]:
saida_video = cv2.VideoWriter(nome_arquivo, fourcc, fps, (video_largura, video_altura))

In [23]:
threshold = 0.5
threshold_NMS = 0.3
fonte_pequena, fonte_media = 0.4, 0.6
fonte = cv2.FONT_HERSHEY_SIMPLEX

In [24]:
amostras_exibir = 20
amostra_atual = 0

In [25]:
cars = 0

## Processamento do vídeo e exibição do resultado

In [26]:
while (cv2.waitKey(1) < 0):
  conectado, frame = cap.read() # Lendo o frame do vídeo
  if not conectado:
    break
  t = time.time()
  frame = cv2.resize(frame, (video_largura, video_altura))
  try:
    (H, W) = frame.shape[:2]
  except:
    print('Erro')
    continue

  # Resultados da rede neural
  imagem_cp = frame.copy() 
  net, frame, layerOutputs = blob_imagem(net, frame)
  caixas = []       
  confiancas = []   
  IDclasses = []  
  detec = []  

  # Informações dos objetos detectados
  for output in layerOutputs:
    for detection in output:
      caixas, confiancas, IDclasses = deteccoes(detection, threshold, caixas, confiancas, IDclasses)

  objs = cv2.dnn.NMSBoxes(caixas, confiancas, threshold, threshold_NMS) # Reduzir bounding box mantendo somente o com maior probabilidade

  # Destacando objetos através da criação dos retângulos e informações
  if len(objs) > 0:
    for i in objs.flatten():
      frame, x, y, w, h = funcoes_imagem(frame, i, confiancas, caixas, COLORS, LABELS, mostrar_texto=False)
      objeto = imagem_cp[y:y + h, x:x + w]

      centro = pega_centro(x,y,w,h)
      detec.append(centro)
      cv2.circle(frame, centro, 4, (0,0,255), -1)

      for (x,y) in detec:
        if y<(pos_linha+offset) and y>(pos_linha-offset):
          cars+=1
          cv2.line(frame, (25, pos_linha), (1200, pos_linha), (0,127,255), 3)
          detec.remove((x,y))
          print("Car is detected: "+str(cars))
  
  cv2.putText(frame, " frame processado em {:.2f} segundos".format(time.time() - t), 
              (20, video_altura-20), fonte, fonte_pequena, (250, 250, 250), 0, lineType=cv2.LINE_AA)

  cv2.putText(frame, "VEHICLE COUNTING: "+str(cars), (450,70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,0), 5)


  
  if amostra_atual <= amostras_exibir:
    plt.imshow(frame)
    amostra_atual += 1

  saida_video.write(frame)

print('Terminou')
saida_video.release()
cv2.destroyAllWindows()

YOLO levou 2.35 segundos


TypeError: imshow() missing required argument 'mat' (pos 2)